In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=3dad8aa21fbc9a1681b1bfd3c44d2e6ead715ba599fedc4d8c2b7af1fae4b19c
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
#tempViews


from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.functions import *

# Create a Spark session
spark = SparkSession.builder.appName("PandasToPySparkWithSchema").getOrCreate()

# Define the schema so it matches with the Big Query later
# NOTE: Please adjust the column names and types according to your CSV's columns.
schema = StructType([
    StructField("CustomerID", IntegerType(), nullable=False),
    StructField("PersonID", IntegerType(), nullable=True),
    StructField("StoreID", FloatType(), nullable=True),
    StructField("TerritoryID", IntegerType(), nullable=True),
    StructField("AccountNumber", StringType(), nullable=True),
    StructField("rowguid", StringType(), nullable=True),
    StructField("ModifiedDate", TimestampType(), nullable=True)
])



df_customer = spark.read.csv('table_customer.csv', sep=',', encoding='utf-8', header=True, schema=schema)
df_person = spark.read.csv('table_person.csv', sep=',', encoding='utf-8', header=True)
#--migue person
#df_person_2 = spark.read.csv('migue_Person.csv', sep=',', encoding='utf-8', header=True)
df_soh = spark.read.csv('table_soh_new.csv', sep=',', encoding='utf-8', header=True)


df_customer.createOrReplaceTempView("customer")

df_person.createOrReplaceTempView("person")

df_soh.createOrReplaceTempView("soh")




df_personid_customer = spark.sql("""

    SELECT c.CustomerID,
           soh.CustomerID AS PersonID,
           c.StoreID,
           c.TerritoryID,
           c.AccountNumber,
           c.rowguid,
           c.ModifiedDate

    FROM customer AS c
    Inner JOIN  soh
      ON c.CustomerID = soh.CustomerID



""")


In [ ]:

df_personid_customer = df_personid_customer.select(
    col("CustomerID").cast(IntegerType()).alias("CustomerID"),
    col("PersonID").cast(IntegerType()).alias("PersonID"),
    col("StoreID").cast(FloatType()).alias("StoreID"),
    col("TerritoryID").cast(IntegerType()).alias("TerritoryID"),
    col("AccountNumber").cast(StringType()).alias("AccountNumber"),
    col("rowguid").cast(TimestampType()).alias("rowguid"),
)
df_personid_customer.printSchema()
#df_personid_customer.write.csv('table_customer_person_id.csv', header=True, mode='overwrite')

root
 |-- CustomerID: integer (nullable = true)
 |-- PersonID: integer (nullable = true)
 |-- StoreID: float (nullable = true)
 |-- TerritoryID: integer (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- rowguid: timestamp (nullable = true)



In [ ]:
#DataViz new table customer test

df_new = spark.read.csv('table_customer_personid_new.csv', sep=',', encoding='utf-8', header=True)

In [ ]:
df_new.printSchema()
print()
df_new.show()

root
 |-- CustomerID: string (nullable = true)
 |-- PersonID: string (nullable = true)
 |-- StoreID: string (nullable = true)
 |-- TerritoryID: string (nullable = true)
 |-- AccountNumber: string (nullable = true)
 |-- rowguid: string (nullable = true)


+----------+--------+-------+-----------+-------------+-------+
|CustomerID|PersonID|StoreID|TerritoryID|AccountNumber|rowguid|
+----------+--------+-------+-----------+-------------+-------+
|     29485|   29485|  294.0|          4|   AW00029485|   null|
|     29485|   29485|  294.0|          4|   AW00029485|   null|
|     29486|   29486|  296.0|          3|   AW00029486|   null|
|     29486|   29486|  296.0|          3|   AW00029486|   null|
|     29486|   29486|  296.0|          3|   AW00029486|   null|
|     29486|   29486|  296.0|          3|   AW00029486|   null|
|     29486|   29486|  296.0|          3|   AW00029486|   null|
|     29486|   29486|  296.0|          3|   AW00029486|   null|
|     29486|   29486|  296.0|          3|